In [ ]:

try:
    import peft  # noqa: F401
except ImportError:
    %pip -q install peft accelerate transformers datasets sentencepiece


In [ ]:

from peft import LoraConfig, get_peft_model, PeftModel

def enable_lora(model, task_type="CAUSAL_LM", r=16, alpha=32, dropout=0.05):
    """Навешивает LoRA-адаптеры на распространённые слои GPT-2/KoGPT2.
    Возвращает PEFT-модель (model), готовую к обучению.
    """

    target_modules = ["c_attn", "c_proj", "c_fc", "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    cfg = LoraConfig(
        r=r,
        lora_alpha=alpha,
        lora_dropout=dropout,
        bias="none",
        task_type=task_type,
        target_modules=target_modules,
    )
    peft_model = get_peft_model(model, cfg)
    try:
        peft_model.print_trainable_parameters()
    except Exception as e:
        print("LoRA enabled (trainable params shown may be limited):", e)
    return peft_model


In [ ]:

try:
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
except NameError:
    pass


In [ ]:
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

In [ ]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

In [ ]:
import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy


In [ ]:
import json, random
from datasets import Dataset

data_path = 'KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path, 'r', encoding='utf-8-sig') as f:
    triplets = json.load(f)



In [ ]:
def triplet_to_pairs(ex):
    pairs = []
    comps = [ex['completion_0'], ex['completion_1'], ex['completion_2']]
    ranks = ex['ranking']
    idx = [0,1,2]
    for a in range(3):
        for b in range(a+1,3):
            i,j = idx[a], idx[b]
            if ranks[i] < ranks[j]:
                chosen, rejected = comps[i], comps[j]
            else:
                chosen, rejected = comps[j], comps[i]
            pairs.append({
                "prompt": ex["prompt"],
                "chosen": chosen,
                "rejected": rejected
            })
    return pairs

pairs = []
for ex in triplets:
    pairs.extend(triplet_to_pairs(ex))

def ok(s): return isinstance(s,str) and len(s.strip())>3
pairs = [p for p in pairs if ok(p["prompt"]) and ok(p["chosen"]) and ok(p["rejected"])]


random.seed(42)
random.shuffle(pairs)
split = int(0.9*len(pairs))
train_data = pairs[:split]
eval_data  = pairs[split:]

print("train:", len(train_data), "eval:", len(eval_data))

In [ ]:
from transformers import AutoTokenizer
INSTR = "### Instruction:\n"
RESP  = "\n\n### Response:\n"

base_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    base_id,
    bos_token="</s>", eos_token="</s>", unk_token="</s>", pad_token="</s>",
    padding_side="right", model_max_length=512
)

def format_sample(prompt, completion):
    text = f"{INSTR}{prompt}{RESP}{completion}{tokenizer.eos_token}"
    return text

class RMPairsDataset:
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.data)
    def __getitem__(self, i):
        ex = self.data[i]
        return {
            "chosen_text":   format_sample(ex["prompt"], ex["chosen"]),
            "rejected_text": format_sample(ex["prompt"], ex["rejected"])
        }

train_ds = RMPairsDataset(train_data, tokenizer)
eval_ds  = RMPairsDataset(eval_data, tokenizer)

import torch
class RMDataCollator:
    def __init__(self, tokenizer, max_len=512):
        self.tok = tokenizer
        self.max_len = max_len
    def __call__(self, batch):
        chosen  = [b["chosen_text"]   for b in batch]
        rejected= [b["rejected_text"] for b in batch]
        enc_ch = self.tok(chosen,   return_tensors="pt", padding=True, truncation=True, max_length=self.max_len)
        enc_rj = self.tok(rejected, return_tensors="pt", padding=True, truncation=True, max_length=self.max_len)
        return {
            "chosen_input_ids": enc_ch["input_ids"],
            "chosen_attention_mask": enc_ch["attention_mask"],
            "rejected_input_ids": enc_rj["input_ids"],
            "rejected_attention_mask": enc_rj["attention_mask"],
        }

collator = RMDataCollator(tokenizer)


In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config

class GPT2RewardModel(nn.Module):
    def __init__(self, pretrained_id="skt/kogpt2-base-v2", gradient_checkpointing=False):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained(pretrained_id)
        if gradient_checkpointing:
            self.backbone.gradient_checkpointing_enable()
        hidden = self.backbone.config.n_embd
        self.value_head = nn.Linear(hidden, 1)

    def forward(self, input_ids, attention_mask):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = out.last_hidden_state
        lengths = attention_mask.sum(dim=1) - 1  # [B]
        pooled = last_hidden[torch.arange(input_ids.size(0)), lengths]  # [B,H]
        reward = self.value_head(pooled).squeeze(-1)  # [B]
        return reward


def rm_step(model, batch, optimizer=None, device="cuda"):
    model = model.to(device)
    for k in batch:
        batch[k] = batch[k].to(device)
    r_ch = model(batch["chosen_input_ids"],   batch["chosen_attention_mask"])
    r_rj = model(batch["rejected_input_ids"], batch["rejected_attention_mask"])

    diff = r_ch - r_rj
    loss = -torch.log(torch.sigmoid(diff)).mean()
    if optimizer:
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    with torch.no_grad():
        acc = (r_ch > r_rj).float().mean().item()
    return loss.item(), acc


In [ ]:
# === Включаем LoRA на только что созданной модели ===
try:
    model  # noqa: F821
    _scope_model = model
except NameError:
    try:
        _scope_model = base_model  # noqa: F821
    except NameError:
        try:
            _scope_model = actor_model  # noqa: F821
        except NameError:
            try:
                _scope_model = policy  # noqa: F821
            except NameError:
                try:
                    _scope_model = rm_model  # noqa: F821
                except NameError:
                    try:
                        _scope_model = backbone  # noqa: F821
                    except NameError:
                        _scope_model = None

if _scope_model is not None:
    # Пытаемся определить тип задачи (по классу); для GPT2 это CAUSAL_LM ок.
    task_type = "CAUSAL_LM"
    model = enable_lora(_scope_model, task_type=task_type, r=16, alpha=32, dropout=0.05)
else:
    print("Предупреждение: не удалось найти переменную модели для навешивания LoRA.")


In [ ]:
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
rm_model = GPT2RewardModel(pretrained_id=base_id, gradient_checkpointing=True).to(device)

opt = torch.optim.AdamW(rm_model.parameters(), lr=2e-5, weight_decay=0.01)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collator)
eval_loader  = DataLoader(eval_ds,  batch_size=8, shuffle=False, collate_fn=collator)

from tqdm.auto import tqdm
import time

steps_per_epoch = len(train_loader)
total_steps = steps_per_epoch * 2

for epoch in range(2):
    rm_model.train()
    tot_loss = tot_acc = 0
    t0 = time.time()

    pbar = tqdm(enumerate(train_loader), total=steps_per_epoch, desc=f"Epoch {epoch+1} [train]")
    for step, batch in pbar:
        loss, acc = rm_step(rm_model, batch, optimizer=opt, device=device)
        tot_loss += loss
        tot_acc  += acc

        avg_loss = tot_loss / (step + 1)
        avg_acc  = tot_acc / (step + 1)

        pbar.set_postfix({"loss": f"{avg_loss:.4f}", "acc": f"{avg_acc:.3f}"})

    rm_model.eval()
    eval_loss = eval_acc = 0
    with torch.no_grad():
        pbar = tqdm(enumerate(eval_loader), total=len(eval_loader), desc=f"Epoch {epoch+1} [eval]")
        for step, batch in pbar:
            loss, acc = rm_step(rm_model, batch, optimizer=None, device=device)
            eval_loss += loss
            eval_acc  += acc

            avg_loss = eval_loss / (step + 1)
            avg_acc  = eval_acc / (step + 1)

            pbar.set_postfix({"loss": f"{avg_loss:.4f}", "acc": f"{avg_acc:.3f}"})



In [ ]:
from pathlib import Path

SAVE_DIR = Path.cwd() / "output_RM"   # Или Path.home() / "output_RM"
SAVE_DIR.mkdir(parents=True, exist_ok=True)


rm_model.backbone.save_pretrained(SAVE_DIR)

tokenizer.save_pretrained(SAVE_DIR)

torch.save(rm_model.value_head.state_dict(), os.path.join(SAVE_DIR, "value_head.bin"))

print("✅ RM сохранена в:", SAVE_DIR)


In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
rm_model.eval()
rm_model.to(device)

eval_loader_pairs = DataLoader(eval_ds, batch_size=32, shuffle=False, collate_fn=RMDataCollator(tokenizer))

def tensor1d(x):
    if isinstance(x, tuple): x = x[0]
    x = x.squeeze(-1)
    return x

tot, correct, margins = 0, 0, []
with torch.no_grad():
    for batch in eval_loader_pairs:
        ch_ids = batch["chosen_input_ids"].to(device)
        ch_ms  = batch["chosen_attention_mask"].to(device)
        rj_ids = batch["rejected_input_ids"].to(device)
        rj_ms  = batch["rejected_attention_mask"].to(device)

        r_ch = tensor1d(rm_model(ch_ids, ch_ms)).cpu().numpy()  # [B]
        r_rj = tensor1d(rm_model(rj_ids, rj_ms)).cpu().numpy()  # [B]
        diff = r_ch - r_rj
        margins.extend(diff.tolist())
        correct += (diff > 0).sum()
        tot += diff.shape[0]

pair_acc = correct / tot
print(f"[Pairwise] Acc={pair_acc:.3f} | mean_margin={np.mean(margins):.3f} | median_margin={np.median(margins):.3f}")


In [ ]:
import numpy as np

def score_candidates(prompt, candidates, tokenizer, rm_model, max_len=512, topk=None):
    rm_model.eval()
    texts = [format_sample(prompt, c) for c in candidates]
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
    with torch.no_grad():
        rewards = tensor1d(rm_model(enc["input_ids"].to(device), enc["attention_mask"].to(device))).cpu().numpy()
    order = np.argsort(-rewards)
    print(f"\n[Prompt]\n{prompt}\n")
    for rank, idx in enumerate(order[:topk] if topk else order, 1):
        print(f"#{rank}  reward={rewards[idx]:.3f}\n{candidates[idx]}\n")
    return rewards, order


my_prompt = "서울에서 주말에 아이와 갈 만한 실내 체험 활동 추천해줘"
my_candidates = [

    "서울 어린이대공원 동물원이나 놀이시설도 추천해요. 입장료가 저렴해서 가족 단위 방문객이 많습니다.",
    "날씨가 좋으면 잠실 롯데월드타워 전망대, 실내활동이면 국립중앙박물관 어린이박물관도 좋아요.",
    "아이 연령대에 따라 다르지만, 5~7세라면 코엑스 키자니아 같은 체험형 공간이 적합합니다.",

    "서울에는 다양한 체험 공간이 있어요. 인터넷 검색해보시면 많은 정보를 얻을 수 있습니다.",
    "추천드릴 수는 없지만 서울에는 어린이들을 위한 좋은 장소가 많습니다.",
    "아이와 함께라면 실내 전시관이나 놀이시설이 무난합니다.",


    "저는 인공지능이기 때문에 추천이 불가능합니다.",
    "알려드릴 수 없습니다.",
    "모릅니다."
]
_ = score_candidates(my_prompt, my_candidates, tokenizer, rm_model, max_len=512, topk=None)


기존 GPTRM_custom에서 단순히 GPT2Model + value_head 구조였는데, 지금은 GPT2RewardModel과 LoRA/gradient checkpointing 같은 설정을 적용 가능하게 수정.

In [ ]:
import shutil


folder = "output_RM"

shutil.make_archive("output_RM_backup", 'zip', folder)